# Deploy machine learning models to Azure

description: (preview) deploy your machine learning or deep learning model as a web service in the Azure cloud.

## Connect to your workspace

In [ ]:
from azureml.core import Workspace

# get workspace configurations
ws = Workspace.from_config()

# get subscription and resourcegroup from config
SUBSCRIPTION_ID = ws.subscription_id
RESOURCE_GROUP = ws.resource_group

RESOURCE_GROUP, SUBSCRIPTION_ID

In [ ]:
!az account set -s $SUBSCRIPTION_ID
!az ml workspace list --resource-group=$RESOURCE_GROUP

## Register your model

A registered model is a logical container stored in the cloud, containing all files located at `model_path`, which is associated with a version number and other metadata.



## Register a model from a local file

You can register a model by providing the local path of the model. You can provide the path of either a folder or a single file on your local machine.

In [ ]:
!wget https://aka.ms/bidaf-9-model -o model.onnx
!az ml model register -n bidaf_onnx -p ./model.onnx

## Deploy your machine learning model

Replace bidaf_onnx:1 with the name of your model and its version number

In [ ]:
!az ml model deploy -n myservice -m bidaf_onnx:1 --overwrite --ic dummyinferenceconfig.json --dc deploymentconfig.json
!az ml service get-logs -n myservice

## Call into your model

In [ ]:
!curl -v http://localhost:32267
!curl -v -X POST -H "content-type:application/json" -d '{"query": "What color is the fox", "context": "The quick brown fox jumped over the lazy dog."}' http://localhost:32267/score

Notice the use of the AZUREML_MODEL_DIR environment variable to locate your registered model. Now that you've added some pip packages, you also need to update your inference configuration with [new configurations](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=azcli#tabpanel_7_azcli) to add in those additional packages

## Deploy again and call your service

Now that we've deployed successfully with a dummy entry script, let's try deploying with a real one. Replace `bidaf_onnx:1` with the name of your model and its version number

In [ ]:
!az ml model deploy -n myservice -m bidaf_onnx:1 --overwrite --ic inferenceconfig.json --dc deploymentconfig.json
!az ml service get-logs -n myservice

Then ensure you can send a post request to the service:

In [ ]:
!curl -v -X POST -H "content-type:application/json" -d '{"query": "What color is the fox", "context": "The quick brown fox jumped over the lazy dog."}' http://localhost:32267/score

## Re-deploy to cloud

Once you've confirmed your service works locally and chosen a remote compute target, you are ready to deploy to the cloud.
Change your re-deploy configuration to correspond to the compute target you've chosen, in this case Azure Container Instances.

Deploy your service again

In [ ]:
!az ml model deploy -n myaciservice -m bidaf_onnx:1 --overwrite --ic inferenceconfig.json --dc re-deploymentconfig.json
!az ml service get-logs -n myaciservice

## Call your remote webservice

When you deploy remotely, you may have key authentication enabled. The example below shows how to get your service key with Python in order to make an inference request.

In [ ]:
import requests
import json
from azureml.core import Webservice, Workspace

ws = Workspace.from_config()

service = Webservice(workspace=ws, name="myaciservice")
scoring_uri = service.scoring_uri

# If the service is authenticated, set the key or token
key, _ = service.get_keys()

# Set the appropriate headers
headers = {"Content-Type": "application/json"}
headers["Authorization"] = f"Bearer {key}"

# Make the request and display the response and logs
data = {
    "query": "What color is the fox",
    "context": "The quick brown fox jumped over the lazy dog.",
}
data = json.dumps(data)
resp = requests.post(scoring_uri, data=data, headers=headers)
print(resp.text)

In [ ]:
print(service.get_logs())

# Delete resources 

In [ ]:
# Get the current model id
import os

stream = os.popen(
    'az ml model list --model-name=bidaf_onnx --latest --query "[0].id" -o tsv'
)
MODEL_ID = stream.read()[0:-1]
MODEL_ID

In [ ]:
!az ml service delete -n myservice
!az ml service delete -n myaciservice
!az ml model delete --model-id=$MODEL_ID

## Next Steps

Try reading [our documentation](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python)